In [2]:
from dataloader import MidiDataModule
from model import MusicPerformer
import pytorch_lightning as pl

c:\Users\gersa\.conda\envs\compute3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
hparams = {"version": 1.0, ## change that on each new training
           "batch_size": 8,
           "data_path": "data",
           "tokenizer": "REMI", 
           "num_workers": 1,
           "pitch_range": range(21,109),
           "beat_res":  {(0,4):8, (4,12):4},
           "prepare_data": False,
           "nb_velocities": 32,
           "additional_tokens": {'Chord': True, 'Rest': True, 'Tempo': True, 'Program': False,
                                  'rest_range': (2, 8),  # (half, 8 beats)
                                  'nb_tempos': 32,  # nb of tempo bins
                                  'tempo_range': (40, 250),
                                  'TimeSignature': False},
           "sequence_len": 512,
            #### Model Related
           "hidden_size": 256,
           "num_encoders": 12,
           "num_heads": 8,
           "learning_rate": 1e-3,
           "weight_decay": 1e-4,
           "vocab_size": None, # no need to touch this one
            ### Generation of Sequences,
           "repetition_penalty": 0.0,
           "temperature": 1.2,
           "seq_len_generate": 100, ##how many tokens to generate
           "init_len_generate": 20 ## how many initial tokens provide to generator 
            }

In [4]:
data = MidiDataModule(hparams=hparams)
#data.prepare_data() # only once
data.setup()
hparams["vocab_size"] = len(data.tokenizer.vocab.token_to_event.keys())
print("Vocabulary size:", hparams["vocab_size"])

Vocabulary size: 277


## Train Model and Save a Checkpoint

In [5]:
model = MusicPerformer(hparams=hparams)
trainer = pl.Trainer(max_epochs=20, gpus=1, limit_val_batches = 1, limit_train_batches= 100)
trainer.fit(model, data)
trainer.save_checkpoint("%s_trained.ckpt" %hparams["version"])

unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory ine

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
Missing logger folder: d:\Code\music\sound_graph\lightning_logs


unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version
unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type                  | Params
------------------------------------------------------
0 | transformer | AutoregressiveWrapper | 15.9 M
------------------------------------------------------
15.9 M    Trainable params
0         Non-trainable params
15.9 M    Total params
63.596    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\gersa\.conda\envs\compute3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

## Generate sample from model checkpoint

In [ ]:
sample = data.val_dataset[1] ##specify ID of the sample to use 
model = MusicPerformer.load_from_checkpoint("path to checkpoint")
model.generate_sequence(input_sequence=sample, output_filename="sample", num_tokens_to_use=50, num_token_to_generate=100,
                        temperature = 1.0, 
                        repetition_penalty = 0.0,
                        filter_thres = 0.9,        ## parameter for the top_k
                        filter_logits_fn="top_k" ) ## can use "top_p"